In [ ]:
!pip install twikit

In [ ]:
import pandas as pd
import asyncio
import random
import os
import nest_asyncio 
from dotenv import load_dotenv
from twikit import Client

In [ ]:
nest_asyncio.apply()
load_dotenv
USERNAME = os.getenv("USERNAME")
EMAIL = os.getenv("EMAIL")
PASSWORD = os.getenv("PASSWORD")
client = Client('en-US')

In [ ]:
async def search_and_collect_tweets(query, THROTTLE_MIN = 10, THROTTLE_MAX = 20, MAX_PAGES=30, MAX_TWEETS=100):
    COOKIE_FILE = 'cookies.json'
    tweet_data = []

    # Login / Cookie 
    try:
        if os.path.exists(COOKIE_FILE):
            client.load_cookies(COOKIE_FILE)
            print("Loaded existing cookies")
        else:
            await client.login(
                auth_info_1=USERNAME,
                auth_info_2=EMAIL,
                password=PASSWORD
            )
            client.save_cookies(COOKIE_FILE)
            print("Saved new cookies")
    except Exception as e:
        print(f"Login failed: {str(e)}")
        return pd.DataFrame()

    # Fungsi utk search
    page = await client.search_tweet(query=query, product='Latest', count=20)

    page_count = 0

    # Selama ada page
    while page:
        page_count += 1
        print(f"Processing page {page_count} (current total tweets: {len(tweet_data)})")

        for tweet in page:
            tweet_data.append({
                'created_at': tweet.created_at,
                'text': tweet.text
            })

        # Check stopping conditions BEFORE fetching next page
        if len(tweet_data) >= MAX_TWEETS:
            print(f"Reached {len(tweet_data)} tweets; stopping.")
            break
        if page_count >= MAX_PAGES:
            print(f"Fetched {page_count} pages; stopping.")
            break

        # Throttle 10-20 seconds between pages
        sleep_secs = random.uniform(THROTTLE_MIN, THROTTLE_MAX)
        print(f"Sleeping for {sleep_secs:.1f}s before next page...")
        await asyncio.sleep(sleep_secs)

        # Try to fetch the next page and break if none or error
        try:
            page = await page.next()
            if not page:
                print("No more pages; stopping.")
                break
        except Exception as e:
            print(f"Pagination stopped (blocked or error): {e}")
            break

    df = pd.DataFrame(tweet_data)

    return df



In [ ]:
# make date = today in format yyyy-mm-dd
today = pd.to_datetime('today').strftime('%Y-%m-%d')

In [ ]:
THROTTLE_MIN = 10 # seconds
THROTTLE_MAX = 20 # seconds
MAX_PAGES = 30 # Rate limit 50 per 15 minutes.
MAX_TWEETS = 100 # Rate limit 150 per 15 minutes.
sincedate = '2025-05-08'
untildate = '2025-07-14'
query = "pantai sanur bali"
query_today = f"pantai sanur bali since:{today}"
query_sincedate = f"pantai sanur bali since:{sincedate} until:{today}"
query_untildate = f"pantai sanur bali until:{untildate}"
# Format since until contoh: pantai sanur until:2025-07-14 since:2025-05-08
# Kirim query ke fungsi search_and_collect_tweets tergantung kebutuhan.

In [ ]:
async def main():
    df_tweets = await search_and_collect_tweets(query_today, THROTTLE_MIN, THROTTLE_MAX, MAX_PAGES, MAX_TWEETS)
    return df_tweets

In [ ]:
df_tweets = await main()

In [ ]:
len(df_tweets)

In [ ]:
df_tweets

Login failed: status: 400, message: "{"errors":[{"code":399,"message":"Salah. Silakan coba lagi. g;174669037387682407:-1746690374114:CUWhxbPJ3s1ZdIaxKQ0wDgvu:6"}]}"
Collected 0 tweets
Empty DataFrame
Columns: []
Index: []


In [ ]:
df_tweets.to_csv('tweets_data.csv', index=False)